In [8]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'LM(영어)', '번역(한-영)', '번역(한-러)', '번역(한-스)', '다국어 번역 교정']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
text_set = set()
langs =  ['ko-KR', 'es-ES', 'en-US', 'ru-RU', 'en-GB']
nationality_set = set()

#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            source_language = line['S-Code']    ## all ko-KR
            source_text = line['원문'].replace('  ', ' ').strip()
            if edited_source_text:=line['수정원문'] != 'N/A':
                source_text = edited_source_text.replace('  ', ' ').strip()
            
            if source_text in text_set:
                continue
            text_set.add(source_text)
            
            target_language = line['T-Code']
            preference_order = ['MT', '1차수정', '2차수정', '최종번역문', '수정번역문']
            target_texts = ['']
            for pref in preference_order:
                text = str(line[pref]).replace('  ', ' ').strip()
                if text != 'N/A' and target_texts[-1] != text:
                    target_texts.append(text)
            target_texts = list(reversed(target_texts))[:-1]
            target_nationality = line['T-Nationality']
            #### data preprocess end 
            
            #### LM(한국어)
            data = {'text': source_text}
            ## preprocess data from line
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end
            
            
            if target_language == 'en-US':
                #### LM(영어)
                data = {'text': target_texts[0]}
                ## preprocess data from line
                
                ## preprocess data from line end
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(영어) end
            
            data = {'input': {}, 'output': None}
            data['input']['source_text'] = source_text
            data['input']['source_language'] = source_language
            data['input']['target_language'] = target_language
            data['input']['target_natinality'] = target_nationality
            data['output'] = target_texts[0]
            
            #### 번역(한-영)
            if target_language in ['en-US', 'en-GB']:
                task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-영) end
            
            
            #### 번역(한-러)
            if target_language == 'ru-RU':
                task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-러) end
            
            
            #### 번역(한-스)
            if target_language == 'es-ES':
                task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-스) end
            
            
            if len(target_texts) > 1:
                data['output'] = target_texts
                #### 다국어 번역 교정
                task_files[5].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 다국어 번역 교정 end
            
            
            # break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 21/21 [00:01<00:00, 15.55it/s]


In [9]:
import pickle

## save text_set
with open('text_set.pkl', 'wb') as f:
    pickle.dump(text_set, f)